In [13]:
import os

train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


In [14]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D
from keras.layers import Activation,Dropout,Flatten,Dense
from keras.optimizers import RMSprop

def build_simple_convnet(input_shape=(224,224,3),class_num=10):
    
    model = Sequential()
    model.add(Conv2D(32,(3,3),input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Conv2D(128,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(class_num))
    model.add(Activation('softmax'))
    
#     rmsprop = RMSprop(lr=10e-3)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
from keras.preprocessing import image

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=5,
                                         horizontal_flip=True,
                                         fill_mode='nearest')
validation_datagen = image.ImageDataGenerator(rescale=1./255)

batch_size = 16
train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=(224,224),
                                    batch_size=batch_size,
                                    shuffle=True,
                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=(224,224),
                                    batch_size=batch_size,
                                    shuffle=False,
                                    class_mode='categorical')


Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [ ]:
from keras.callbacks import TensorBoard

model = build_simple_convnet()
model.fit_generator(train_generator,
                    steps_per_epoch=18047 // batch_size,
                    epochs=3,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=4377 // batch_size)

Epoch 1/3
1127/1127 [==============================] - 3890s 3s/step - loss: 1.9015 - acc: 0.2967 - val_loss: 2.1572 - val_acc: 0.2999
Epoch 2/3
1127/1127 [==============================] - 3818s 3s/step - loss: 1.2567 - acc: 0.5433 - val_loss: 2.7569 - val_acc: 0.3210
Epoch 3/3
 258/1127 [=====>........................] - ETA: 48:42 - loss: 1.0378 - acc: 0.6289